In [ ]:
%pip install load_dotenv 

In [ ]:
import requests
from dotenv import load_dotenv
import os

# 加载最新的 .env 文件
load_dotenv(override=True)

# auth_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
upload_url_template = 'https://graph.microsoft.com/v1.0/users/me@lvhongyuan.site/drive/root:/{}:/content'

def get_access_token(client_id, client_secret, auth_url):
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    data = {
        'client_id': client_id,
        'scope': 'https://graph.microsoft.com/.default',
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.post(auth_url, headers=headers, data=data)
    response.raise_for_status()
    token = response.json().get('access_token')
    return token

def upload_file_to_onedrive(access_token, file_path, upload_url):
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/octet-stream'
    }
    with open(file_path, 'rb') as file:
        response = requests.put(upload_url, headers=headers, data=file)
    response.raise_for_status()
    return response.json()

def main():

    # 设置参数
    localFilePath = os.getenv('localFilePath') 
    oneDrivePath = os.getenv('oneDrivePath')  # 文件在 OneDrive 中的路径和名称
    client_id = os.getenv('client_id')
    client_secret = os.getenv('client_secret')
    tenant_id = os.getenv('tenant_id')  # 通常是目录 (tenant) ID

    # file_path = 'hosts'
    # file_name = '文档/Backup/Semaphore/hosts'  # 文件在 OneDrive 中的路径和名称

    auth_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    upload_url = upload_url_template.format(oneDrivePath)

    print('获取访问令牌...')
    access_token = get_access_token(client_id, client_secret, auth_url)
    print(f'访问令牌获取成功。{access_token}')

    print('上传文件到 OneDrive...')
    response = upload_file_to_onedrive(access_token, localFilePath, upload_url)
    print('文件上传成功。')
    print('响应:', response)

if __name__ == "__main__":
    main()


In [ ]:
from dotenv import load_dotenv
import os

# 加载最新的 .env 文件
load_dotenv(override=True)

# 设置参数
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
tenant_id = os.getenv('tenant_id')  # 通常是目录 (tenant) ID
oneDrivePath = os.getenv('oneDrivePath')
localFilePath = os.getenv('localFilePath')

!echo $localFilePath
!echo $oneDrivePath
!echo $client_id
!echo $client_secret
!echo $tenant_id

!sh ./backup.sh $localFilePath $oneDrivePath $client_id $client_secret $tenant_id